In [ ]:
import numpy as np
import pandas as pd
import requests
import math
import pandas_datareader.data as web
import datetime as dt
from scipy import stats
import matplotlib.pyplot as plt
from statsmodels.regression.rolling import RollingOLS
import yfinance as yf

import warnings
warnings.filterwarnings("ignore")


In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=c9e013a3a00d38336429f7249ebe16b6576fc7e9fa33e72d12596e90e642dfd5
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
import pandas_ta

In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.1 MB/s eta 0:00:00


In [ ]:
import xlsxwriter as xl

In [ ]:
sp500=pd.read_csv('/content/SP500.csv')
sp500['Symbol']=sp500['Symbol'].str.replace('.','-')

symbols_list=sp500['Symbol'].unique().tolist()
symbols_list


['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [ ]:
end_date='2023-09-27'
start_date=pd.to_datetime(end_date)-pd.DateOffset(365*8)
df=yf.download(tickers=symbols_list,start=start_date,end=end_date).stack()
df.index.names=['date','ticker']
df.columns=df.columns.str.lower()
df

[*********************100%%**********************]  503 of 503 completed
ERROR:yfinance:
3 Failed downloads:
ERROR:yfinance:['SOLV', 'GEV', 'VLTO']: YFChartError("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.483564   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651131   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334896   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547684  176.869995  178.449997  176.270004  176.580002   

Price                   volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAL       7478800.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[987967 rows x 6 columns]

In [ ]:
df['german_klass_vol']=((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)
df['rsi']=df.groupby(level=1)['adj close'].transform(lambda x:pandas_ta.rsi(close=x, length=20))
df['bb_low']=df.groupby(level=1)['adj close'].transform(lambda x:pandas_ta.bbands(close=np.log1p(x),Length=20).iloc[:,0])
df['bb_mid']=df.groupby(level=1)['adj close'].transform(lambda x:pandas_ta.bbands(close=np.log1p(x),Length=20).iloc[:,1])
df['bb_high']=df.groupby(level=1)['adj close'].transform(lambda x:pandas_ta.bbands(close=np.log1p(x),Length=20).iloc[:,2])
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.483564   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651131   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334896   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547684  176.869995  178.449997  176.270004  176.580002   

Price                   volume  german_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000998        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054656        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146750  4.487736   
           YUM       1500600.0         -0.000051  36.057179  4.806831   
           ZBH       3610500.0          0.000022  31.893229  4.718433   
           ZBRA       355400.0          0.000133  29.494977  5.403153   
           ZTS       1463200.0          0.000062  42.623451  5.158698   

Price                bb_mid   bb_high  
date       ticker                      
2015-09-29 A            NaN       NaN  
           AAL          NaN       NaN  
           AAPL         NaN       NaN  
           ABBV         NaN       NaN  
           ABT          NaN       NaN  
...                     ...       ...  
2023-09-26 XYL     4.516030  4.544324  
           YUM     4.822895  4.838960  
           ZBH     4.761870  4.805306  
           ZBRA    5.436866  5.470580  
           ZTS     5.179178  5.199658  

[987967 rows x 11 columns]

In [ ]:
def compute_atr(stock_data):
  return pandas_ta.atr(high=stock_data['high'],
              low=stock_data['low'],
              close=stock_data['close'],
              length=14).sub(pandas_ta.atr(high=stock_data['high'],
              low=stock_data['low'],
              close=stock_data['close'],
              length=14).mean()).div(pandas_ta.atr(high=stock_data['high'],
              low=stock_data['low'],
              close=stock_data['close'],
              length=14).std())
df['atr']=df.groupby(level=1,group_keys=False).apply(compute_atr)
df


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.483564   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651131   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334896   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547684  176.869995  178.449997  176.270004  176.580002   

Price                   volume  german_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000998        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054656        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146750  4.487736   
           YUM       1500600.0         -0.000051  36.057179  4.806831   
           ZBH       3610500.0          0.000022  31.893229  4.718433   
           ZBRA       355400.0          0.000133  29.494977  5.403153   
           ZTS       1463200.0          0.000062  42.623451  5.158698   

Price                bb_mid   bb_high       atr  
date       ticker                                
2015-09-29 A            NaN       NaN       NaN  
           AAL          NaN       NaN       NaN  
           AAPL         NaN       NaN       NaN  
           ABBV         NaN       NaN       NaN  
           ABT          NaN       NaN       NaN  
...                     ...       ...       ...  
2023-09-26 XYL     4.516030  4.544324  0.033800  
           YUM     4.822895  4.838960  0.142547  
           ZBH     4.761870  4.805306 -0.381708  
           ZBRA    5.436866  5.470580 -0.057389  
           ZTS     5.179178  5.199658  0.651515  

[987967 rows x 12 columns]

In [ ]:
def compute_macd(close):
  macd =pandas_ta.macd(close=close,length=20).iloc[:,0]
  return macd.sub(macd.mean()).div(macd.std())
df['macd']=df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.483564   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651131   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334896   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547684  176.869995  178.449997  176.270004  176.580002   

Price                   volume  german_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000998        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054656        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146750  4.487736   
           YUM       1500600.0         -0.000051  36.057179  4.806831   
           ZBH       3610500.0          0.000022  31.893229  4.718433   
           ZBRA       355400.0          0.000133  29.494977  5.403153   
           ZTS       1463200.0          0.000062  42.623451  5.158698   

Price                bb_mid   bb_high       atr      macd  
date       ticker                                          
2015-09-29 A            NaN       NaN       NaN       NaN  
           AAL          NaN       NaN       NaN       NaN  
           AAPL         NaN       NaN       NaN       NaN  
           ABBV         NaN       NaN       NaN       NaN  
           ABT          NaN       NaN       NaN       NaN  
...                     ...       ...       ...       ...  
2023-09-26 XYL     4.516030  4.544324  0.033800 -2.159189  
           YUM     4.822895  4.838960  0.142547 -1.363695  
           ZBH     4.761870  4.805306 -0.381708 -0.881067  
           ZBRA    5.436866  5.470580 -0.057389 -1.600791  
           ZTS     5.179178  5.199658  0.651515 -1.188279  

[987967 rows x 13 columns]

In [ ]:
df['dollar_volume']=(df['adj close']*df['volume'])/1e6
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.483564   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651131   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334896   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547684  176.869995  178.449997  176.270004  176.580002   

Price                   volume  german_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000998        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054656        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146750  4.487736   
           YUM       1500600.0         -0.000051  36.057179  4.806831   
           ZBH       3610500.0          0.000022  31.893229  4.718433   
           ZBRA       355400.0          0.000133  29.494977  5.403153   
           ZTS       1463200.0          0.000062  42.623451  5.158698   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      70.913580  
           AAL          NaN       NaN       NaN       NaN     279.420098  
           AAPL         NaN       NaN       NaN       NaN    7234.160250  
           ABBV         NaN       NaN       NaN       NaN     466.641803  
           ABT          NaN       NaN       NaN       NaN     411.369604  
...                     ...       ...       ...       ...            ...  
2023-09-26 XYL     4.516030  4.544324  0.033800 -2.159189     117.344880  
           YUM     4.822895  4.838960  0.142547 -1.363695     183.389847  
           ZBH     4.761870  4.805306 -0.381708 -0.881067     402.696470  
           ZBRA    5.436866  5.470580 -0.057389 -1.600791      79.595386  
           ZTS     5.179178  5.199658  0.651515 -1.188279     256.861371  

[987967 rows x 14 columns]

In [ ]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume','close', 'open','high','volume','low']]
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'), df.unstack()[last_cols].resample('M').last().stack('ticker')], axis=1)).dropna()
data


dollar_volume   adj close       atr   bb_high    bb_low  \
date       ticker                                                            
2015-11-30 A          135.992831   39.023186 -1.033887  3.708485  3.639405   
           AAL        287.915789   39.429928  0.190822  3.725534  3.687555   
           AAPL      4023.983766   26.854136 -0.967900  3.332486  3.319084   
           ABBV       337.563911   40.393528 -0.526809  3.791759  3.719999   
           ABT        211.659051   38.293564 -1.064842  3.688688  3.671040   
...                          ...         ...       ...       ...       ...   
2023-09-30 OTIS       154.361755   78.356499 -1.028320  4.411154  4.372395   
           ABNB      1633.500725  132.279999 -1.006939  4.936271  4.872183   
           CEG        196.670369  107.862030 -0.436215  4.727086  4.684423   
           GEHC       212.275850   66.130219 -0.893478  4.221833  4.160434   
           KVUE       670.804304   20.006193 -0.899746  3.061936  3.037789   

                     bb_mid  german_klass_vol      macd        rsi  
date       ticker                                                   
2015-11-30 A       3.673945         -0.001992  0.567156  73.421477  
           AAL     3.706544         -0.000966 -0.418773  40.719003  
           AAPL    3.325785         -0.003307 -0.142790  55.537336  
           ABBV    3.755879         -0.059549  0.145676  49.376867  
           ABT     3.679864         -0.011216  0.335556  56.962527  
...                     ...               ...       ...        ...  
2023-09-30 OTIS    4.391775         -0.000097 -1.534536  33.116224  
           ABNB    4.904227          0.000213 -0.037854  44.494127  
           CEG     4.705754          0.000131  0.366876  55.245485  
           GEHC    4.191133          0.000185 -1.116463  40.922336  
           KVUE    3.049862         -0.000159 -1.435621  35.706312  

[46648 rows x 9 columns]

In [ ]:
data['dollar_volume']=(data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())
data['dollar_vol_rank']=(data.groupby('date')['dollar_volume'].rank(ascending=False))
data=data[data['dollar_vol_rank']<150].drop(['dollar_volume','dollar_vol_rank'],axis=1)
data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2020-10-31 AAL      11.280000 -1.244464  2.553427  2.462650  2.508038   
           AAPL    106.506020  1.409489  4.768817  4.666705  4.717761   
           ABBV     73.725426 -0.194581  4.326853  4.240976  4.283915   
           ABT      98.835159  0.983090  4.650643  4.588146  4.619394   
           ACN     206.515442  0.515540  5.358882  5.316237  5.337559   
...                       ...       ...       ...       ...       ...   
2023-09-30 VRTX    351.690002  0.029799  5.874662  5.856530  5.865596   
           VZ       31.258692 -1.078816  3.493420  3.470153  3.481786   
           WFC      39.781090 -0.558742  3.770301  3.696784  3.733542   
           WMT      53.597538 -0.196381  4.009767  3.993212  4.001489   
           XOM     113.372101  0.601335  4.746444  4.721125  4.733784   

                   german_klass_vol      macd        rsi  
date       ticker                                         
2020-10-31 AAL             0.000622 -0.348861  41.027057  
           AAPL            0.000078 -0.628203  43.770334  
           ABBV           -0.004957 -0.994069  47.517640  
           ABT            -0.001040 -0.069481  48.047145  
           ACN            -0.000481 -0.949113  42.280828  
...                             ...       ...        ...  
2023-09-30 VRTX            0.000037  0.027907  52.406728  
           VZ             -0.000940 -0.350386  42.222480  
           WFC            -0.000110 -0.282325  40.920296  
           WMT            -0.000041  0.399459  54.722541  
           XOM            -0.000065  1.400623  59.440190  

[5364 rows x 8 columns]

In [ ]:
def calculate_returns(df):
    outlier_cutoff = 0.005
    lags = [1, 2, 3, 6, 9, 12]
    for lag in lags:
        df[f'return_{lag}m'] = (
            df['adj close']
            .pct_change(lag)
            .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff), upper=x.quantile(1 - outlier_cutoff)))
            .add(1)
            .pow(1 / lag)
            .sub(1)
        )
    return df

data = df.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
data


Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-11-03 A        35.710602   38.270000   38.520000   38.160000   38.310001   
           AAL      44.437527   46.500000   47.090000   46.080002   47.020000   
           AAPL     27.704834   30.642500   30.872499   30.174999   30.197500   
           ABBV     43.366604   62.430000   63.720001   61.919998   63.119999   
           ABT      38.796543   45.509998   45.639999   45.040001   45.360001   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736298   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211014  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547684  176.869995  178.449997  176.270004  176.580002   

Price                   volume  german_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-11-03 A         1485800.0         -0.001863  71.847370  3.574942   
           AAL       6983200.0         -0.000998  71.197087  3.795021   
           AAPL    182076000.0         -0.002606  70.791257  3.322311   
           ABBV      8754700.0         -0.054013  67.525856  3.586901   
           ABT       3933800.0         -0.009349  76.628696  3.662437   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146750  4.487736   
           YUM       1500600.0         -0.000051  36.057179  4.806831   
           ZBH       3610500.0          0.000022  31.893229  4.718433   
           ZBRA       355400.0          0.000133  29.494977  5.403153   
           ZTS       1463200.0          0.000062  42.623451  5.158698   

Price                bb_mid   bb_high       atr      macd  dollar_volume  \
date       ticker                                                          
2015-11-03 A       3.595308  3.615673 -1.163852  0.477838      53.058812   
           AAL     3.807925  3.820828  0.752390  2.675407     310.316136   
           AAPL    3.341466  3.360620 -0.914141  0.192340    5044.385353   
           ABBV    3.727265  3.867629  0.688900  0.519548     379.661607   
           ABT     3.675972  3.689508 -0.938211  1.134168     152.617841   
...                     ...       ...       ...       ...            ...   
2023-09-26 XYL     4.516030  4.544324  0.033800 -2.159189     117.344880   
           YUM     4.822895  4.838960  0.142547 -1.363695     183.389847   
           ZBH     4.761870  4.805306 -0.381708 -0.881067     402.696470   
           ZBRA    5.436866  5.470580 -0.057389 -1.600791      79.595386   
           ZTS     5.179178  5.199658  0.651515 -1.188279     256.861371   

Price              return_1m  return_2m  return_3m  return_6m  return_9m  \
date       ticker                                                          
2015-11-03 A       -0.008292   0.006731   0.005015   0.006413   0.007129   
           AAL      0.002804   0.004106   0.005352   0.000467   0.005615   
           AAPL     0.011470   0.012764   0.005610   0.010272   0.008322   
           ABBV    -0.014989   0.023896   0.034743   0.026338   0.016605   
           ABT      0.001761   0.007893   0.001985   0.007133   0.008426   
...                      ...        ...        ...        ...        ...   
2023-09-26 XYL     -0.016156  -0.007620  -0.006976  -0.010685  -0.006592   
           YUM     -0.007682  -0.003968  -0.001821  -0.004706  -0.004434   
           ZBH     -0.041670  -0.018965  -0.015345  -0.013519  -0.007762   
           ZBRA    -0.018365  -0.007047  -0.006928  -0.020244  -0.012226   
           ZTS     -0.004783   0.000538   0.000453  -0.002596  -0.003172   

Price             

In [ ]:

factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2015-11-30 A       0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.003574
           AAL     0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.010314
           AAPL    0.0056  0.0331 -0.0042 -0.0275 -0.0099   0.004159
           ABBV    0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.030833
           ABT     0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.011008
...                   ...     ...     ...     ...     ...        ...
2023-08-31 XYL    -0.0239 -0.0365 -0.0106  0.0344 -0.0236   0.000483
           YUM    -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.005534
           ZBH    -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.009315
           ZBRA   -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.014689
           ZTS    -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.011365

[33382 rows x 6 columns]

In [ ]:
observations= factor_data.groupby(level=1).size()
valid_stocks= observations[observations >=10]
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2015-11-30 A       0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.003574
           AAL     0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.010314
           AAPL    0.0056  0.0331 -0.0042 -0.0275 -0.0099   0.004159
           ABBV    0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.030833
           ABT     0.0056  0.0331 -0.0042 -0.0275 -0.0099  -0.011008
...                   ...     ...     ...     ...     ...        ...
2023-08-31 XYL    -0.0239 -0.0365 -0.0106  0.0344 -0.0236   0.000483
           YUM    -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.005534
           ZBH    -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.009315
           ZBRA   -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.014689
           ZTS    -0.0239 -0.0365 -0.0106  0.0344 -0.0236  -0.011365

[33372 rows x 6 columns]

In [ ]:
valid_stocks.index

Index(['A', 'AAL', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI',
       ...
       'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZTS'],
      dtype='object', name='ticker', length=498)

In [ ]:
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

In [ ]:
betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'],
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas


Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2015-11-30 A            NaN       NaN       NaN       NaN       NaN
           AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2023-08-31 XYL     0.096491 -0.104049  0.049621  0.004513 -0.164114
           YUM     0.086325 -0.218433  0.144405 -0.104686 -0.168250
           ZBH     0.151505 -0.166276  0.059833 -0.182181 -0.053888
           ZBRA    0.133309 -0.121560  0.234703 -0.047293 -0.272353
           ZTS     0.052306 -0.120322  0.017653  0.027057 -0.013435

[33372 rows x 5 columns]

In [ ]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

data = (data.join(betas.groupby('ticker').shift(),  lsuffix='_left', rsuffix='_right'))

data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

data = data.drop('adj close', axis=1)

data = data.dropna()

data.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 974146 entries, (Timestamp('2015-11-03 00:00:00'), 'A') to (Timestamp('2023-09-26 00:00:00'), 'ZTS')
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   close             974146 non-null  float64
 1   high              974146 non-null  float64
 2   low               974146 non-null  float64
 3   open              974146 non-null  float64
 4   volume            974146 non-null  float64
 5   german_klass_vol  974146 non-null  float64
 6   rsi               974146 non-null  float64
 7   bb_low            974146 non-null  float64
 8   bb_mid            974146 non-null  float64
 9   bb_high           974146 non-null  float64
 10  atr               974146 non-null  float64
 11  macd              974146 non-null  float64
 12  dollar_volume     974146 non-null  float64
 13  return_1m         974146 non-null  float64
 14  return_2m         974146 non-null  float64


In [ ]:
from sklearn.cluster import KMeans


def get_clusters(df):
    df['cluster'] = KMeans(n_clusters=4,
                           random_state=0,
                           init='random').fit(df).labels_
    return df

data = data.dropna().groupby('date', group_keys=False).apply(get_clusters)

data

close        high         low        open  \
date       ticker                                                   
2015-11-03 A        38.270000   38.520000   38.160000   38.310001   
           AAL      46.500000   47.090000   46.080002   47.020000   
           AAPL     30.642500   30.872499   30.174999   30.197500   
           ABBV     62.430000   63.720001   61.919998   63.119999   
           ABT      45.509998   45.639999   45.040001   45.360001   
...                       ...         ...         ...         ...   
2023-09-26 XYL      89.519997   90.849998   89.500000   90.379997   
           YUM     124.010002  124.739998  123.449997  124.239998   
           ZBH     112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  226.649994  222.580002  225.970001   
           ZTS     176.869995  178.449997  176.270004  176.580002   

                        volume  german_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-11-03 A         1485800.0         -0.001863  71.847370  3.574942   
           AAL       6983200.0         -0.000998  71.197087  3.795021   
           AAPL    182076000.0         -0.002606  70.791257  3.322311   
           ABBV      8754700.0         -0.054013  67.525856  3.586901   
           ABT       3933800.0         -0.009349  76.628696  3.662437   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146750  4.487736   
           YUM       1500600.0         -0.000051  36.057179  4.806831   
           ZBH       3610500.0          0.000022  31.893229  4.718433   
           ZBRA       355400.0          0.000133  29.494977  5.403153   
           ZTS       1463200.0          0.000062  42.623451  5.158698   

                     bb_mid   bb_high  ...  return_3m  return_6m  return_9m  \
date       ticker                      ...                                    
2015-11-03 A       3.595308  3.615673  ...   0.005015   0.006413   0.007129   
           AAL     3.807925  3.820828  ...   0.005352   0.000467   0.005615   
           AAPL    3.341466  3.360620  ...   0.005610   0.010272   0.008322   
           ABBV    3.727265  3.867629  ...   0.034743   0.026338   0.016605   
           ABT     3.675972  3.689508  ...   0.001985   0.007133   0.008426   
...                     ...       ...  ...        ...        ...        ...   
2023-09-26 XYL     4.516030  4.544324  ...  -0.006976  -0.010685  -0.006592   
           YUM     4.822895  4.838960  ...  -0.001821  -0.004706  -0.004434   
           ZBH     4.761870  4.805306  ...  -0.015345  -0.013519  -0.007762   
           ZBRA    5.436866  5.470580  ...  -0.006928  -0.020244  -0.012226   
           ZTS     5.179178  5.199658  ...   0.000453  -0.002596  -0.003172   

                   return_12m    Mkt-RF       SMB       HML       RMW  \
date       ticker                                                       
2015-11-03 A         0.005622 -0.021113 -0.015924  0.078578  0.050368   
           AAL       0.005350  0.003066 -0.355818  0.182684 -0.133521   
           AAPL      0.008267  0.086930 -0.045529 -0.031042  0.131119   
           ABBV      0.008307  0.048118  0.063729 -0.094331  0.195013   
           ABT       0.006273 -0.058297  0.094087  0.027054  0.177240   
...                       ...       ...       ...       ...       ...   
2023-09-26 XYL      -0.005131 -0.044279 -0.014096  0.080655  0.155368   
           YUM      -0.001675 -0.065210 -0.069297  0.162818  0.035448   
           ZBH      -0.005841 -0.024005  0.116072 -0.114641  0.012290   
           ZBRA     -0.011200 -0.029993  0.058322  0.067649  0.091083   
           ZTS      -0.004087  0.018411  0.075045 -0.057897  0.231199   

                        CMA  cluster  
date       ticker                     
2015-11-03 A      -0.010643        0  
           AAL    -0.126518        0  
           AAPL    0.320967        1  
   

In [ ]:

def plot_clusters(data):

    cluster_0 = data[data['cluster']==0]
    cluster_1 = data[data['cluster']==1]
    cluster_2 = data[data['cluster']==2]
    cluster_3 = data[data['cluster']==3]

    plt.scatter(cluster_0.iloc[:,0] , cluster_0.iloc[:,6] , color = 'red', label='cluster 0')
    plt.scatter(cluster_1.iloc[:,0] , cluster_1.iloc[:,6] , color = 'green', label='cluster 1')
    plt.scatter(cluster_2.iloc[:,0] , cluster_2.iloc[:,6] , color = 'blue', label='cluster 2')
    plt.scatter(cluster_3.iloc[:,0] , cluster_3.iloc[:,6] , color = 'black', label='cluster 3')

    plt.legend()
    plt.show()
    return

In [ ]:
plt.style.use('ggplot')

for i in data.index.get_level_values('date').unique().tolist():

    g = data.xs(i, level=0)

    plt.title(f'Date {i}')

    plot_clusters(g)

In [ ]:

target_rsi_values = [30, 45, 55, 70]

initial_centroids = np.zeros((len(target_rsi_values), 18))

initial_centroids[:, 6] = target_rsi_values

initial_centroids

array([[ 0.,  0.,  0.,  0.,  0.,  0., 30.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 45.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 55.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 70.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.]])

In [ ]:
filtered_df = data[data['cluster']==3].copy()

filtered_df = filtered_df.reset_index(level=1)

filtered_df.index = filtered_df.index+pd.DateOffset(1)

filtered_df = filtered_df.reset_index().set_index(['date', 'ticker'])

dates = filtered_df.index.get_level_values('date').unique().tolist()

fixed_dates = {}

for d in dates:

    fixed_dates[d.strftime('%Y-%m-%d')] = filtered_df.xs(d, level=0).index.tolist()

fixed_dates

{'2015-11-04': ['ABBV',
  'ADM',
  'AES',
  'AMAT',
  'AMD',
  'AXON',
  'BSX',
  'C',
  'CMCSA',
  'COP',
  'CSX',
  'CVX',
  'DAL',
  'EBAY',
  'EXC',
  'F',
  'FIS',
  'FITB',
  'FTNT',
  'GE',
  'GLW',
  'GM',
  'HAL',
  'HST',
  'INTC',
  'JPM',
  'KEY',
  'KMI',
  'KO',
  'MET',
  'META',
  'MGM',
  'MO',
  'MRK',
  'MRO',
  'MS',
  'MU',
  'NFLX',
  'NRG',
  'ORCL',
  'PARA',
  'PG',
  'PYPL',
  'QCOM',
  'RF',
  'SBUX',
  'SLB',
  'SYF',
  'T',
  'V',
  'VTRS',
  'VZ',
  'WFC',
  'XOM',
  'ZTS'],
 '2015-11-05': ['AIG',
  'C',
  'CMCSA',
  'CSCO',
  'DIS',
  'EXC',
  'F',
  'FCX',
  'GE',
  'GOOG',
  'GOOGL',
  'HAL',
  'HPQ',
  'INTC',
  'KEY',
  'KMI',
  'META',
  'MSFT',
  'MU',
  'NFLX',
  'ORCL',
  'PARA',
  'PFE',
  'T',
  'WMT',
  'XOM'],
 '2015-11-06': ['ADSK',
  'AES',
  'C',
  'CMCSA',
  'CMG',
  'CSCO',
  'CSX',
  'DIS',
  'EBAY',
  'F',
  'FCX',
  'FTNT',
  'GE',
  'GOOG',
  'GOOGL',
  'HAL',
  'HPQ',
  'INTC',
  'JPM',
  'KEY',
  'KMI',
  'MRO',
  'MS',
  'MSFT',
  

In [ ]:

stocks = data.index.get_level_values('ticker').unique().tolist()

new_df = yf.download(tickers=stocks,
                     start=data.index.get_level_values('date').unique()[0]-pd.DateOffset(months=12),
                     end=data.index.get_level_values('date').unique()[-1])

new_df

[*********************100%%**********************]  496 of 496 completed


Price        Adj Close                                                \
Ticker               A        AAL        AAPL        ABBV        ABT   
Date                                                                   
2014-11-03   37.718056  39.795147   24.312372   42.538895  36.137444   
2014-11-04   37.108513  40.458405   24.134588   42.142097  36.404579   
2014-11-05   37.062321  40.032021   24.192368   42.061382  36.446316   
2014-11-06   38.207558  40.875298   24.261555   42.175720  36.471355   
2014-11-07   37.801182  40.932140   24.330751   41.361927  36.204224   
...                ...        ...         ...         ...        ...   
2023-09-19  111.713280  13.190000  178.364563  149.137848  99.142769   
2023-09-20  111.872437  13.060000  174.798676  149.254456  98.758682   
2023-09-21  109.146988  13.060000  173.244797  149.079544  97.429169   
2023-09-22  110.818077  12.900000  174.101410  148.409119  96.660995   
2023-09-25  111.345245  12.910000  175.386337  150.264954  95.991318   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2014-11-03  18.799999   69.222717   69.910004   40.419033  36.019501  ...   
2014-11-04  18.910000   69.376602   71.070000   40.199535  37.765190  ...   
2014-11-05  18.906668   70.513626   71.370003   40.671040  38.352165  ...   
2014-11-06  19.026667   71.163399   72.099998   40.825500  38.092983  ...   
2014-11-07  19.160000   71.368561   71.599998   40.841759  38.535137  ...   
...               ...         ...         ...         ...        ...  ...   
2023-09-19  81.199997  313.655060  541.690002  176.375748  77.244774  ...   
2023-09-20  81.320000  313.131195  535.780029  173.741211  77.518211  ...   
2023-09-21  80.290001  308.386749  513.880005  171.530945  77.029938  ...   
2023-09-22  80.959999  312.429443  512.900024  173.149170  75.174507  ...   
2023-09-25  81.150002  313.318970  511.600006  173.099838  75.487000  ...   

Price       Volume                                                         \
Ticker         WTW       WY     WYNN      XEL       XOM      XYL      YUM   
Date                                                                        
2014-11-03  504566  3013800  1222900  3243800  13410000   781100  4277047   
2014-11-04  440467  2763400  2088000  3709800  13498500   693800  3248402   
2014-11-05  309965  3056200  2938200  4556000  12480000   823700  2750842   
2014-11-06  398564  2924200  3081700  8182500  14731400  1218200  3727741   
2014-11-07  369837  2184200  1489600  4070000  13233500   669300  3368446   
...            ...      ...      ...      ...       ...      ...      ...   
2023-09-19  795300  2662600  2036900  7856000  12562900  1695500  1299300   
2023-09-20  670700  2374200  1061300  7670600  11879900  1752100  1437200   
2023-09-21  620600  3494500  2358400  4904800  14400600  2044800  1492700   
2023-09-22  651100  3476300  2611300  4904300  13197300  1699800  1534100   
2023-09-25  477500  2269600  1387800  3617800  11316000  1702000  1048500   

Price                                   
Ticker          ZBH     ZBRA       ZTS  
Date                                    
2014-11-03   813082   833000   4071300  
2014-11-04  1050703  1544000   9723700  
2014-11-05  1231983  1475600  11878900  
2014-11-06   764054   816600   4457100  
2014-11-07   854694   507200   6819000  
...             ...      ...       ...  
2023-09-19   970200   245000   1373600  
2023-09-20  1055900   824900   1485500  
2023-09-21  1489200   676200   1691800  
2023-09-22  1267300   482200   1281800  
2023-09-25   886700   314500   1551100  

[2238 rows x 2976 columns]

In [ ]:
returns_dataframe = np.log(new_df['Adj Close']).diff()

portfolio_df = pd.DataFrame()

for start_date in fixed_dates.keys():

    try:

        end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')

        cols = fixed_dates[start_date]

        optimization_start_date = (pd.to_datetime(start_date)-pd.DateOffset(months=12)).strftime('%Y-%m-%d')

        optimization_end_date = (pd.to_datetime(start_date)-pd.DateOffset(days=1)).strftime('%Y-%m-%d')

        optimization_df = new_df[optimization_start_date:optimization_end_date]['Adj Close'][cols]

        success = False
        try:
            weights = optimize_weights(prices=optimization_df,
                                   lower_bound=round(1/(len(optimization_df.columns)*2),3))

            weights = pd.DataFrame(weights, index=pd.Series(0))

            success = True
        except:
            print(f'Max Sharpe Optimization failed for {start_date}, Continuing with Equal-Weights')

        if success==False:
            weights = pd.DataFrame([1/len(optimization_df.columns) for i in range(len(optimization_df.columns))],
                                     index=optimization_df.columns.tolist(),
                                     columns=pd.Series(0)).T

        temp_df = returns_dataframe[start_date:end_date]

        temp_df = temp_df.stack().to_frame('return').reset_index(level=0)\
                   .merge(weights.stack().to_frame('weight').reset_index(level=0, drop=True),
                          left_index=True,
                          right_index=True)\
                   .reset_index().set_index(['Date', 'index']).unstack().stack()

        temp_df.index.names = ['date', 'ticker']

        temp_df['weighted_return'] = temp_df['return']*temp_df['weight']

        temp_df = temp_df.groupby(level=0)['weighted_return'].sum().to_frame('Strategy Return')

        portfolio_df = pd.concat([portfolio_df, temp_df], axis=0)

    except Exception as e:
        print(e)

portfolio_df = portfolio_df.drop_duplicates()

portfolio_df

"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
"None of ['index'] are in the columns"
Max Sharpe Optimization failed for 2015-12-08, Continuing with Equal-Weights
"None of ['index'] are in the columns"
"None of ['index'] are in 

,Strategy Return
date,
2015-11-30,-0.011681
2015-12-31,-0.011448
2016-01-29,0.023780
2016-02-29,-0.005111
2016-03-31,-0.001274
...,...
2023-05-31,-0.009052
2023-06-30,0.021099
2023-07-31,0.000349


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
def optimize_weights(prices,lower_bound=0):
  returns=expected_returns.mean_historical_return(prices=prices,frequency=252)
  cov= risk_models.sample_cov(prices=prices,frequency=252)
  ef=EfficientFrontier(expected_returns=returns,
                       cov_matrix=cov,
                       weight_bounds=(lower_bound, .1),solver='SCS')
  weights=ef.max_sharpe()
  return ef.clean_weights()

In [ ]:
!pip install PyPortfolioOpt


In [ ]:
stocks=data.index.get_level_values('ticker').unique().tolist()
new_df=yf.download(tickers=stocks,
                     start=data.index.get_level_values('date').unique()[0]-pd.DateOffset(months=12),
                     end=data.index.get_level_values('date').unique()[-1])
new_df

[*********************100%%**********************]  496 of 496 completed


Price        Adj Close                                                \
Ticker               A        AAL        AAPL        ABBV        ABT   
Date                                                                   
2014-11-03   37.718060  39.795151   24.312378   42.538906  36.137447   
2014-11-04   37.108509  40.458405   24.134588   42.142094  36.404575   
2014-11-05   37.062336  40.032024   24.192369   42.061390  36.446316   
2014-11-06   38.207539  40.875305   24.261555   42.175724  36.471363   
2014-11-07   37.801182  40.932152   24.330746   41.361935  36.204216   
...                ...        ...         ...         ...        ...   
2023-09-19  111.713280  13.190000  178.364563  149.137863  99.142776   
2023-09-20  111.872429  13.060000  174.798676  149.254440  98.758682   
2023-09-21  109.146996  13.060000  173.244797  149.079529  97.429169   
2023-09-22  110.818077  12.900000  174.101410  148.409119  96.661003   
2023-09-25  111.345245  12.910000  175.386337  150.264954  95.991318   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2014-11-03  18.799999   69.222710   69.910004   40.419033  36.019485  ...   
2014-11-04  18.910000   69.376617   71.070000   40.199539  37.765186  ...   
2014-11-05  18.906668   70.513641   71.370003   40.671043  38.352177  ...   
2014-11-06  19.026667   71.163376   72.099998   40.825512  38.092991  ...   
2014-11-07  19.160000   71.368553   71.599998   40.841759  38.535130  ...   
...               ...         ...         ...         ...        ...  ...   
2023-09-19  81.199997  313.655060  541.690002  176.375732  77.244781  ...   
2023-09-20  81.320000  313.131195  535.780029  173.741211  77.518211  ...   
2023-09-21  80.290001  308.386810  513.880005  171.530945  77.029938  ...   
2023-09-22  80.959999  312.429413  512.900024  173.149170  75.174507  ...   
2023-09-25  81.150002  313.318970  511.600006  173.099823  75.487000  ...   

Price       Volume                                                         \
Ticker         WTW       WY     WYNN      XEL       XOM      XYL      YUM   
Date                                                                        
2014-11-03  504566  3013800  1222900  3243800  13410000   781100  4277047   
2014-11-04  440467  2763400  2088000  3709800  13498500   693800  3248402   
2014-11-05  309965  3056200  2938200  4556000  12480000   823700  2750842   
2014-11-06  398564  2924200  3081700  8182500  14731400  1218200  3727741   
2014-11-07  369837  2184200  1489600  4070000  13233500   669300  3368446   
...            ...      ...      ...      ...       ...      ...      ...   
2023-09-19  795300  2662600  2036900  7856000  12562900  1695500  1299300   
2023-09-20  670700  2374200  1061300  7670600  11879900  1752100  1437200   
2023-09-21  620600  3494500  2358400  4904800  14400600  2044800  1492700   
2023-09-22  651100  3476300  2611300  4904300  13197300  1699800  1534100   
2023-09-25  477500  2269600  1387800  3617800  11316000  1702000  1048500   

Price                                   
Ticker          ZBH     ZBRA       ZTS  
Date                                    
2014-11-03   813082   833000   4071300  
2014-11-04  1050703  1544000   9723700  
2014-11-05  1231983  1475600  11878900  
2014-11-06   764054   816600   4457100  
2014-11-07   854694   507200   6819000  
...             ...      ...       ...  
2023-09-19   970200   245000   1373600  
2023-09-20  1055900   824900   1485500  
2023-09-21  1489200   676200   1691800  
2023-09-22  1267300   482200   1281800  
2023-09-25   886700   314500   1551100  

[2238 rows x 2976 columns]